In [1]:
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

os.environ['CLMM_MODELING_BACKEND'] = 'nc'

__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import timedelta
from astropy import units
from numpy import random
plt.rcParams['text.latex.preamble'] = [r'\usepackage{pxfonts, mathpazo}']
plt.rcParams['font.family']=['Palatino']
plt.rc('text', usetex=True)


import clmm.dataops as da
import clmm.theory as theory
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.utils import convert_units
from scipy.stats import chi2
from scipy.stats import kstest
from scipy.stats import shapiro
import corner
import math

In [2]:
cluster_m = 1.e15
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)
concentration = 4

cluster_m     = 1.e15 # Cluster mass
cluster_z     = 0.4   # Cluster redshift
concentration = 4     # Concentrion parameter NFW profile
ngals         = 10000 # Number of galaxies
Delta         = 200   # Overdensity parameter definition NFW profile
cluster_ra    = 0.0   # Cluster right ascension
cluster_dec   = 0.0   # Cluster declination

def create_mock_cluster(shapenoise, redshift, nbins, method=None):

    if type(redshift) == float or type(redshift) == str:
        data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, redshift, zsrc_min = cluster_z + 0.1, shapenoise=shapenoise, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
    else:
        data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, redshift[0], zsrc_min = cluster_z + 0.1, shapenoise=shapenoise, photoz_sigma_unscaled=redshift[1], ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)

    gc   = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, data)

    if method:
        bin_edges = da.make_bins(0.7, 4, nbins, method=method)
    else:
        bin_edges = da.make_bins(0.7, 4, nbins)

    gc.compute_tangential_and_cross_components(geometry="flat")
    gc.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
    
    return gc

In [3]:
redshift = [("chang13", 0.05)]
nbins    = [10, 50, 100]
method   = [None, "evenlog10width"]
nruns    = 10

for z in redshift:
    for n in nbins:

        if n == 10:
            repeat = 10
        elif n == 50:
            repeat = 2
        else:
            repeat = 1

        for m in method:

            for r in range(repeat):

                shapiro_data = np.empty(shape=(n*nruns, 3))

                for i in range(n):

                    for j in range(nruns):

                        gc = create_mock_cluster(1.e-3, z, n, m)

                        theta    = [gc.galcat['theta'][gal_id] for gal_id in gc.profile[i]['gal_id']]
                        source_z = [gc.galcat['z'][gal_id] for gal_id in gc.profile[i]['gal_id']]
                        radius   = convert_units(theta, 'radians', 'Mpc', redshift=gc.z, cosmo=cosmo)
                        shear    = clmm.compute_reduced_tangential_shear(radius, cluster_m, concentration, gc.z, source_z, cosmo, delta_mdef=200, halo_profile_model='nfw')

                        N = len(theta)

                        shapiro_data[j+nruns*i,0], shapiro_data[j+nruns*i,1] = shapiro(shear)
                        shapiro_data[j+nruns*i,2] = N

                np.save(f'Data/Shapiro/shapiro_data_({z},{n},{m},{r},{nruns})', shapiro_data)